# More Latent Exploring

In [8]:
# imports
%matplotlib inline

from importlib import reload
import os
import numpy as np
import datetime

import pandas

import h5py
import seaborn as sns
import pandas as pd
import umap

from ulmo import io as ulmo_io
from ulmo import plotting

from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from matplotlib import pyplot as plt
from tqdm import trange

# Load latents

In [3]:
### Load latents from s3
latents_last_file = 's3://modis-l2/modis_latents_simclr/modis_latents_last.h5'
with ulmo_io.open(latents_last_file, 'rb') as f:
    ll_hf = h5py.File(f, 'r')    
latents_last = ll_hf['modis_latents'][:]

In [4]:
latents_last.shape

(134162, 128)

# Load MODIS table

In [5]:
modis_tbl = ulmo_io.load_main_table('s3://modis-l2/Tables/MODIS_L2_std.parquet')

Read main table: s3://modis-l2/Tables/MODIS_L2_std.parquet


In [6]:
modis_tbl.head()

,lat,lon,col,datetime,LL,row,clear_fraction,mean_temperature,Tmin,Tmax,T90,T10,filename,UID,pp_file,pp_root,field_size,pp_type,pp_idx
0,13.980967,62.675774,517,2003-03-10 21:25:00,520.178711,304,0.000000,26.199476,25.250000,26.885000,26.474998,25.814999,AQUA_MODIS.20030310T212506.L2.SST.nc,1047103981024267577,s3://modis-l2/PreProc/MODIS_R2019_2003_95clear...,standard,128,0,0
1,13.043748,-102.751816,706,2003-12-12 08:30:00,432.766724,1696,0.049805,28.924648,27.789999,29.654999,29.154999,28.695000,AQUA_MODIS.20031212T083006.L2.SST.nc,107110304377724818,s3://modis-l2/PreProc/MODIS_R2019_2003_95clear...,standard,128,0,1
2,-18.086618,40.844833,581,2003-09-24 22:35:00,764.413086,656,0.006104,25.551611,24.775000,26.154999,25.750000,25.344999,AQUA_MODIS.20030924T223506.L2.SST.nc,106471913422084483,s3://modis-l2/PreProc/MODIS_R2019_2003_95clear...,standard,128,0,2
3,39.071575,18.569141,1028,2003-05-01 01:05:00,371.310120,898,0.049988,16.414951,15.160000,17.205000,16.725000,16.080000,AQUA_MODIS.20030501T010506.L2.SST.nc,1051129071619856914,s3://modis-l2/PreProc/MODIS_R2019_2003_95clear...,standard,128,0,3
4,17.901926,90.155480,997,2003-12-05 19:55:00,-214.759277,1520,0.035461,26.744499,24.975000,28.724998,27.824999,25.830000,AQUA_MODIS.20031205T195506.L2.SST.nc,1070107901927015548,s3://modis-l2/PreProc/MODIS_R2019_2003_95clear...,standard,128,0,4


## Cut down to 2010 + train

In [17]:
year_2010 = (modis_tbl.datetime >= datetime.datetime(2010,1,1)) & (modis_tbl.datetime < datetime.datetime(2011,1,1))

In [18]:
train = modis_tbl.pp_type == 1

In [19]:
modis_tbl[train].head()

,lat,lon,col,datetime,LL,row,clear_fraction,mean_temperature,Tmin,Tmax,T90,T10,filename,UID,pp_file,pp_root,field_size,pp_type,pp_idx
12358049,20.111063,-67.497110,805,2010-08-06 06:25:00,NaN,304,0.027649,29.125587,27.699999,30.695000,30.029999,28.584999,AQUA_MODIS.20100806T062508.L2.SST.nc,1281110111111250289,s3://modis-l2/PreProc/MODIS_R2019_2010_95clear...,standard,128,1,0
12358050,-2.129871,46.129204,773,2010-12-16 22:35:00,NaN,272,0.036377,27.745419,26.779999,28.420000,27.980000,27.514999,AQUA_MODIS.20101216T223508.L2.SST.nc,129287870122612920,s3://modis-l2/PreProc/MODIS_R2019_2010_95clear...,standard,128,1,1
12358051,1.553729,57.486620,581,2010-12-24 21:45:00,NaN,112,0.042175,27.745102,26.359999,28.529999,28.059999,27.369999,AQUA_MODIS.20101224T214500.L2.SST.nc,129391553723748662,s3://modis-l2/PreProc/MODIS_R2019_2010_95clear...,standard,128,1,2
12358052,-12.398787,42.314100,592,2010-07-25 22:35:00,NaN,1600,0.049988,25.856106,23.799999,26.740000,26.170000,25.484999,AQUA_MODIS.20100725T223507.L2.SST.nc,128077601222231410,s3://modis-l2/PreProc/MODIS_R2019_2010_95clear...,standard,128,1,3
12358053,8.551046,103.123344,965,2010-04-22 19:00:00,NaN,1584,0.016968,30.994106,29.209999,32.044998,31.429998,30.484999,AQUA_MODIS.20100422T190000.L2.SST.nc,127198551028312334,s3://modis-l2/PreProc/MODIS_R2019_2010_95clear...,standard,128,1,4


## Need LL for training data!